<p style="text-align:center;font-family: Arial; font-size:3.0em; font-style:bold"><br>
Silent Disco Glamping Network
</p><br>
<p style="text-align:center;font-family: Arial; font-size:2.0em; font-style:bold"><br>
Le Minh Bien
</p><br>
<p style="text-align:center; font-family: Arial; font-size:1.00em;color:white; font-style:bold">Originally created for: </p>
<p style="text-align:center; font-family: Arial; font-size:1.00em;color:green; font-style:bold">
Project 2 of Mathematical Optimisation: Theory and Application, AMSI Summer School 2026</p>

## Problem formulation
Given a set of potential locations $\mathcal{L}$ (set of airstrips in Australia), $\mathcal{L} = {1, 2, ..., m}$.

Each element $i \in \mathcal{L}$ corresponds to one location point in $\mathbb{R}^2$, where each point is indexed by $l_i = (x_i, y_i) \in \mathbb{R}^2$.

#### Parameters:
* $d_{ij} = ||l_i-l_j||_2$: Euclidean distance between location $i$ and $j$ $(d_{ij} = 0)$
* $w_i >0$: weighted cost factor of location $i$
* $\alpha_i$: maximum number of tents at location $i$
* $k$: number of sites to select
* $\delta$: minimum distance between any two selected sites
* $N$: total number of tents across all sites

#### Decision variable
* $x_i \in \{0,1\}$:
$$
x_i = 
\begin{cases}
1, \text{ if location} i \text{is selected,} \\
0, \text{ otherwise}
\end{cases}
$$

#### Mathematical formulation
$$
\begin{aligned}
\max_{x}\quad 
& \frac{\sum_{i,j \in \mathcal{L}} d_{ij} x_i x_j}{\sum_{i \in \mathcal{L}} w_i x_i} \\[6pt]
\text{s.t.}\quad 
& \sum_{i \in \mathcal{L}} x_i = k \\[4pt]
& x_i + x_j \le 1 \quad \forall (i,j): d_{ij} < \delta \\[4pt]
& \sum_{i \in \mathcal{L}} \alpha_i x_i \ge N \\[4pt]
& x_i \in \{0,1\}
\end{aligned}
$$

Let t = $\sum_{i=1}^mw_ix_i (t>0)$, hence, t is a continuous variable

Then the objective becomes $\max_x \frac{\sum_{i,j \in \mathcal{L}}^md_{ij}x_ix_j}{t}$

Let $f(x) = \sum_{i,j \in \mathcal{L}}^md_{ij}x_ix_j$, the problem becomes $max_x \frac{f(x)}{t}$

Since $f(x)$ is quadratic, $f(\frac{x}{t}) = \frac{1}{t^2}f(x)$
Therefore, maximising $\frac{f(x)}{t}$ is equivalent to maximising $g(x,t) = tf(\frac{x}{t})$

From the Max-sum Diversity Problem, Spiers, Bui and Loxton (2023) have proved that $f$ is concave over the feasible set $K = \{x|\sum_{x=1}^m = k\}$. The other constraints of our problem only shrink the feasible set of the Max-sum Diversity problem, hence, do not change the concavity of $f$ on that set.

Since $f$ is proved to be concave, $g(x,t)$ is concave as a perspective mapping of function $f$.

Now, our problem is proved to be concave, we can solve it using Outer Approximation.

In [1]:
# Import necessary libraries
import numpy as np
import gurobipy as gp
from gurobipy import Model, GRB
import math
from typing import List, Tuple
from Linearisationtechnique import qp_model, second_linear
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
import plotly
from numba import njit
import plotly.graph_objects as go
from termcolor import colored
import warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=NumbaPendingDeprecationWarning)
# plotly.graph_objs._scattergl.Scattergl

Create functions to generate distance matrix from locations data

In [ ]:
# calculate distance
@njit
def distance(u: np.ndarray, v: np.ndarray) -> float:
    """
    Calculates the Euclidean distance between two points in a 2D space.

    Args:
        u: A numpy array representing the coordinates of the first point.
        v: A numpy array representing the coordinates of the second point.

    Returns:
        The Euclidean distance between the two points.
    """
    # The formula for Euclidean distance is sqrt(sum((u_i - v_i)^2))
    return np.sqrt(np.sum((u - v) ** 2))


@njit
def distance_matrix(locations: List[np.ndarray]) -> np.ndarray:
    """
    Computes a symmetric distance matrix for a list of locations.

    Args:
        locations: A list or array of coordinates, where each element
                  represents a point in 2D space.

    Returns:
        A symmetric n x n numpy array where the element (i, j) is the
        Euclidean distance between location i and location j.
    """
    # Get the number of locations
    n = len(locations)

    # Initialize an n x n matrix with zeros
    matrix = np.zeros((n, n))

    # Iterate through each pair of locations to compute the distance
    for i in range(n):
        # Start from i + 1 to calculate only the upper triangle
        for j in range(i + 1, n):
            # Calculate the distance between point i and point j
            dist = distance(locations[i], locations[j])
            # Assign the distance to both (i, j) and (j, i) for symmetry
            matrix[i, j] = dist
            matrix[j, i] = dist

    return matrix

## Build the Outer Approximatio model
Introduce a scalar variable $\theta \in \mathbb{R}$ to represent the objective value
The Outer Approximation then replaces $\max g(x,t)$ by $\max\theta\quad \text{s.t.}\ \theta \le g(x,t)$, and then approximates $g$ by supporting hyperplances.

### Compute the gradient of $g(x,t)$
Let $z = \frac{x}{t}$. Since $f$ is quadratic,
$$
\nabla f(x) = Qz
$$
where $Q = [q_{ij}]$ is the distance matrix and $Q_{ii} = 0 \quad \forall i$

Now, 
$$
\nabla_xg(x,t) = \nabla f(\frac{x}{t}) = Q(\frac{x}{t})
$$

Hence,
$$
\begin{aligned}
\frac{\partial g}{\partial t}
&= f\!\left(\frac{x}{t}\right)
  + t \left\langle \nabla f\!\left(\frac{x}{t}\right),
  \frac{\partial}{\partial t}\!\left(\frac{x}{t}\right) \right\rangle \\[6pt]
&= f\!\left(\frac{x}{t}\right)
  + t \left\langle \nabla f\!\left(\frac{x}{t}\right),
  \left(-\frac{x}{t^{2}}\right) \right\rangle \\[6pt]
&= f\!\left(\frac{x}{t}\right)
  - \left\langle \nabla f\!\left(\frac{x}{t}\right),
  \frac{x}{t} \right\rangle \\[6pt]
\frac{\partial g}{\partial t}(x,t)
= f\!\left(\frac{x}{t}\right)
- \left\langle Q\!\left(\frac{x}{t}\right), \frac{x}{t} \right\rangle\\[6pt]
\end{aligned}
$$



### Supporting hyperplane
Let $(y,s)$ be a feasible point with $s>0$.

The supporting hyperplane of g at $(y,s)$ is
$$
h_{(y,s)}(x,t)
= g(y,s)
+ \langle \nabla_x g(y,s),\, x - y \rangle
+ \frac{\partial g}{\partial t}(y,s)\,(t - s).
$$
$$
\begin{aligned}
h_{(y,s)}(x,t)
&= g(y,s)
+ \left\langle Q\!\left(\frac{y}{s}\right),\, x - y \right\rangle \\[6pt]
&\quad + \left[
f\!\left(\frac{y}{s}\right)
- \left\langle Q\!\left(\frac{y}{s}\right),\, \frac{y}{s} \right\rangle
\right](t - s).
\end{aligned}
$$

Now, since $g$ is concave, $g(x,t) \le h_{(y,s)}(x,t) \quad \forall (x,t)$.

Therefore, our Outer Approximation model becomes
$$
\begin{aligned}
\max \quad & \theta \\[4pt]
\text{s.t.} \quad
& \sum_{i=1}^{m} x_i = k, \\[4pt]
& t = \sum_{i=1}^{m} w_i x_i, \\[4pt]
& x_i + x_j \le 1, \quad \forall (i,j): d_{ij} < \delta, \\[4pt]
& \sum_{i=1}^{m} a_i x_i \ge N, \\[4pt]
& \theta \le g(y,s)
+ \langle \nabla_x g(y,s),\, x - y \rangle
+ \frac{\partial g}{\partial t}(y,s)\,(t - s),
\quad \forall (y,s) \in \mathcal{C}, \\[4pt]
& x_i \in \{0,1\}, \quad \theta \le \mathrm{UB}.
\end{aligned}
\qquad (\mathrm{OA})
$$


### Input data and setup model